## Capture tracebacks in IPython/Jupyter

In [ ]:
#| default_exp capture_trace

In [ ]:
#| export
import re
from IPython import get_ipython
from friendlly.utils import nict

In [ ]:
#| export
class SaveTraceback:
    def __init__(self, saved):
        self._saved_showtraceback = saved
        self.tracebacks = []

    def _showtraceback(self, etype, value, stb):
        self.tracebacks.append( nict(
            ename=etype,
            evalue=", ".join(value.args),
            traceback= stb # ["\n".join([ strip_junk(s) for s in stb]) if isinstance(stb, list) else stb]
        ))
        self._saved_showtraceback(etype, value, stb)

    def register(self):
        get_ipython()._showtraceback = self._showtraceback

    def unregister(self):
        get_ipython()._showtraceback = self._saved_showtraceback


In [ ]:
ip = get_ipython()
saver = SaveTraceback(ip._showtraceback)
ip._showtraceback = saver._showtraceback

In [ ]:
#| eval: false
def outer_exception():
    try:
        1 / 0  # Raises ZeroDivisionError
    except ZeroDivisionError:
        raise ValueError("This is a nested exception")

outer_exception()

ValueError: This is a nested exception

In [ ]:
saver.tracebacks

[{'ename': ValueError,
  'evalue': 'This is a nested exception',
  'traceback': ['\x1b---------------------------------------------------------------------------\x1b',
   '\x1bZeroDivisionError\x1b                         Traceback (most recent call last)',
   'Cell \x1bIn[8], line 4\x1b, in \x1bouter_exception\x1b()\x1b\n\x1b      3\x1b \x1btry\x1b:\n\x1b----> 4\x1b     \x1b1\x1b\x1b \x1b\x1b/\x1b\x1b \x1b\x1b0\x1b  \x1b# Raises ZeroDivisionError\x1b\n\x1b      5\x1b \x1bexcept\x1b \x1bZeroDivisionError\x1b:\n',
   '\x1bZeroDivisionError\x1b: division by zero',
   '\nDuring handling of the above exception, another exception occurred:\n',
   '\x1bValueError\x1b                                Traceback (most recent call last)',
   'Cell \x1bIn[8], line 8\x1b\n\x1b      5\x1b     \x1bexcept\x1b \x1bZeroDivisionError\x1b:\n\x1b      6\x1b         \x1braise\x1b \x1bValueError\x1b(\x1b"\x1b\x1bThis is a nested exception\x1b\x1b"\x1b)\n\x1b----> 8\x1b \x1bouter_exception\x1b\x1b(\x1b\x1b)\x1